In [315]:
import gurobipy as gp
import numpy as np
import pandas as pd
from datetime import datetime
from math import floor
import json

In [316]:
starting_time = datetime.now()
time_limit = 5*60

Read data

In [317]:
import re

def parse_dzn(file_path):
    data = {}
    with open(file_path, 'r') as file:
        content = file.read()
        
        # Find all key-value pairs
        key_value_pairs = re.findall(r'(\w+)\s*=\s*(.*?);', content, re.DOTALL)
        
        for key, value in key_value_pairs:
            # Remove any extra whitespace and newlines
            value = value.strip().replace('\n', '')

            #Parse matrix
            if key == 'D':
                rows = value.split('|')
                D = []
                for i in range(1,len(rows)-1):
                    D.append([int(v.strip()) for v in rows[i][1:-1].split(',')])
                value = D
            # Parse arrays
            elif value.startswith('[') and value.endswith(']'):
                value = value[1:-1].split(',')
                value = [int(v.strip()) for v in value]
            else:
                value = int(value)
                
            data[key] = value
    return data

# Usage
'''
file_path = 'inst08.dzn'  # Path to your .dzn file
parsed_data = parse_dzn(file_path)
print(parsed_data)
'''

"\nfile_path = 'inst08.dzn'  # Path to your .dzn file\nparsed_data = parse_dzn(file_path)\nprint(parsed_data)\n"

In [318]:
def read_dat_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Read m and n
    m = int(lines[0].strip())
    n = int(lines[1].strip())
    
    # Read l vector
    l = list(map(int, lines[2].strip().split()))
    
    # Read s vector
    s = list(map(int, lines[3].strip().split()))
    
    # Read D matrix
    D = []
    for line in lines[4:]:
        D.append(list(map(int, line.strip().split())))
    
    return {
        'm': m,
        'n': n,
        'l': l,
        's': s,
        'D': D
    }

Create the model

In [319]:
model = gp.Model()

Parse the instance

In [320]:
innum = input('Type instance number:\n>')
file_path = str(f'inst{innum}.dat')
parsed_data = read_dat_file(file_path)

m = parsed_data['m']
n = parsed_data['n']
s = parsed_data['s']
l = sorted(parsed_data['l'], reverse=True)
D = parsed_data['D']

maxD = np.max(D)

Define the variables

In [321]:
max_total_dist = model.addVar(vtype=gp.GRB.CONTINUOUS, name='MaxTotalDist')

cumulative_dist = model.addVars(m, n+1, vtype=gp.GRB.INTEGER, name='CumulativeDist')
cumulative_load = model.addVars(m, n, lb=0, ub=n*max(s), vtype=gp.GRB.INTEGER, name='CumulativeLoad')

paths = model.addVars(m, n+1, n+1, vtype=gp.GRB.BINARY, name='Paths')

Define Objective function

In [322]:
model.setObjective(max_total_dist, gp.GRB.MINIMIZE)

Define Constraints

In [323]:
#Maximum total distance is equal to the maximum total distance computed in the final depot "slot" for each courier
model.addConstrs( gp.quicksum(cumulative_dist[c,j] for j in range(n+1)) <= max_total_dist for c in range(m))

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>}

In [324]:
#Every node must be visited exactly once (except for the depot n+1)
for dest in range(n):
    model.addConstr(gp.quicksum(paths[c,source,dest] for c in range(m) for source in range(n+1)) <= 1)
    model.addConstr(gp.quicksum(paths[c,source,dest] for c in range(m) for source in range(n+1)) >= 1)
    #model.addConstr(gp.quicksum(paths[c,source,dest] for c in range(m) for source in range(n+1)) == 1)

In [325]:
for c in range(m):
    #Every courier must visit the depot as last node
    model.addConstr(gp.quicksum(paths[c,source,n] for source in range(n)) == 1)

    #Each courier must start from depot
    model.addConstr(gp.quicksum(paths[c,n,dest] for dest in range(n)) == 1)

    for source in range(n+1):
        #Couriers cannot stay still
        model.addConstr(paths[c,source,source] <= 0.5) #npe

        for dest in range(n+1):
            #Cumulative distance update
            model.addConstr((paths[c,source,dest] == 1) >> (cumulative_dist[c,dest] == D[source][dest])) #npe

            #Path contiguity
            model.addConstr((paths[c,source,dest] == 1) >> (gp.quicksum(paths[c,j,source] for j in range(n+1)) >= 1))
            model.addConstr((paths[c,source,dest] == 1) >> (gp.quicksum(paths[c,dest,j] for j in range(n+1)) >= 1))

            #Load update
            if dest < n:
                model.addConstr((paths[c,source,dest] == 1) >> (cumulative_load[c,dest] == s[dest])) #npe

        #Just in order to speed up the search    
        model.addConstr(gp.quicksum(paths[c,source,j] for j in range(n+1)) <= 1)

In [326]:
#Loop avoidance
for c in range(m):
    for source in range(n):
        for dest in range(source+1):
            model.addConstr( (gp.quicksum(paths[c,dest,j] for j in range(source+1)) + paths[c,source,dest]) <= 1)

In [327]:
#Symmetry breaking constraints
for c in range(m-1):
    model.addConstr( gp.quicksum(paths[c,source,dest] for source in range(n) for dest in range(n)) >= \
                     gp.quicksum(paths[c+1,source,dest] for source in range(n) for dest in range(n)) )

In [328]:
#Maximum load
for c in range(m):
    model.addConstr(gp.quicksum(cumulative_load[c,j] for j in range(n)) <= l[c])

In [329]:
preprocessing_time = datetime.now() - starting_time
safe_bound = 5
model.setParam('TimeLimit', time_limit - preprocessing_time.seconds - safe_bound)

2
Set parameter TimeLimit to value 23


In [330]:
model.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 43 rows, 47 columns and 168 nonzeros
Model fingerprint: 0xf9ef7125
Model has 120 general constraints
Variable types: 1 continuous, 46 integer (32 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 6e+01]
  RHS range        [5e-01, 3e+01]
  GenCon rhs range [1e+00, 1e+02]
  GenCon coe range [1e+00, 1e+00]
Presolve removed 43 rows and 47 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 206 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.060000000000e+02, best bound 2.060000000000e+02, gap 0.0000%


In [332]:
def find_path(adj_matrix, courier):
    res = []
    source = n
    res.append(source)
    while True:
        for dest in range(n+1):
            if adj_matrix[courier, source, dest].X == 1:
                res.append(dest)
                source = dest
                break
        if source == n:
            return res

In [ ]:
sol = []
if model.SolCount > 0:
    for c in range(m):
        sol.append(find_path(paths,c))

In [334]:
json_dict = {}
json_dict['time'] = int(floor(model.Runtime + preprocessing_time.seconds)) if model.SolCount > 0 else time_limit
json_dict['optimal'] = True if (model.Runtime + preprocessing_time.seconds + safe_bound < time_limit) else False
json_dict['obj'] = int(model.ObjVal) if model.SolCount > 0 else None
json_dict['sol'] = sol

with open(f'MIP/{str(int(innum))}.json', 'w') as outfile:
    json.dump(json_dict, outfile)